In [6]:
import re


def ints(text: str) -> list[int]:
    return [int(x) for x in re.findall("-?\\d+", text)]


def first(iterable):
    return next(iter(iterable))


def data(day: int, parser=str, sep="\n", example=False) -> list:
    "Split the day's input file into sections separated by `sep`, and apply `parser` to each."
    filename = f"2025/{day}-example.txt" if example else f"2025/{day}.txt"
    sections = open(filename).read().rstrip().split(sep)
    return [parser(section) for section in sections]

In [81]:
# Day 1
rotations = data(1)
dial = 50
password = 0
for rotation in rotations:
    dir = rotation[0]
    steps = int(rotation[1:])
    if dir == "R":
        dial = (dial + steps) % 100
    else:
        dial = (dial - steps) % 100
    if dial == 0:
        password += 1
print(password)  # 995

995


In [83]:
# Day 1, part 2
dial = 50
password = 0
for rotation in rotations:
    dir = rotation[0]
    steps = int(rotation[1:])
    if steps // 100:
        password += steps // 100  # Full rotations
    if dir == "R":
        new_dial = (dial + steps) % 100
        if new_dial < dial:
            password += 1
    else:
        new_dial = (dial - steps) % 100
        if new_dial > dial and dial != 0:
            password += 1
        if new_dial == 0:
            password += 1
    dial = new_dial
print(password)  # 5847

5847


100